<a href="https://colab.research.google.com/github/varshapande/Project-of-machinelearning-/blob/main/frocasting%2C_fillnawith_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np 
from datetime import date, datetime
import time
import calendar
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
train1 = pd.read_csv('/content/drive/MyDrive/forcastingtrain (1).csv')
test = pd.read_csv('/content/drive/MyDrive/forcastingtest.csv')
oil = pd.read_csv('/content/drive/MyDrive/oil.csv')
stores = pd.read_csv('/content/drive/MyDrive/stores.csv')
holidays = pd.read_csv('/content/drive/MyDrive/holidays_events.csv')
transactions = pd.read_csv('/content/drive/MyDrive/transactions.csv.zip')

In [ ]:
train1.shape

In [ ]:
### here train is the concat of train1 and test 

In [ ]:
train = pd.concat([train1,test])

In [ ]:
test.head()

In [ ]:
stores.head()

In [ ]:
holidays.head()

In [ ]:
oil.head()

In [ ]:
merge_data = train.merge(oil, on='date', how='left')
merge_data = merge_data.merge(holidays, on='date', how='left')
merge_data = merge_data.merge(stores, on='store_nbr', how='left')
merge_data = merge_data.merge(transactions, on=['date', 'store_nbr'], how='left')

In [ ]:
merge_data.head()

In [ ]:

merge_data['date'] = pd.to_datetime(merge_data['date']).dt.date
merge_data['year'] = pd.to_datetime(merge_data['date']).dt.year
merge_data['month'] = pd.to_datetime(merge_data['date']).dt.month
merge_data['day'] = pd.to_datetime(merge_data['date']).dt.day

In [ ]:
merge_data.head(20)

In [ ]:
def core(dataset,threshold):
    col = set()
    cormatrix = dataset.corr()
    for i in range(len(cormatrix.columns)):
     for j in range(i):
        if abs(cormatrix.iloc[i,j])>threshold:
             colname = cormatrix.columns[i]
             col.add(colname)
    return col

In [ ]:
corrfeature = core(merge_data,.8)
corrfeature

In [ ]:
### here I am dropping highly corr features if i will not drop i will encode them by doing before str opertaion.

In [ ]:
df = merge_data.drop(columns =['dcoilwtico', 'month', 'year','sales',])

In [ ]:
df.isnull().mean()

In [ ]:
#import calendar
#merge_data['month'] = merge_data['month'].apply(lambda x:calendar.month_abbr[x])

In [ ]:
#merge_data['year']= merge_data['year'].astype(str)

In [ ]:
df['day']= df['day'].astype(str)

In [ ]:
numer = df.select_dtypes(['int64','float64'])

In [ ]:
numermiss = [var for var in numer if numer[var].isnull().sum()>0]
numermiss

In [ ]:
## filling missing value with the same name 
def whole(df,var):
    #mean = df[var].mean()
    df[var] = df[var].fillna(0)
    #median = df[var].median()
    #extreme = df[var].mean()+3*df[var].std()
    #randomsample = df[var].dropna().sample(df[var].isnull().sum(),random_state = 0)
    #df[var]= df[var].fillna(median)
    #df[var] = df[var].fillna(extreme)
    #df[var+'rand']  = df[var]
   # randomsample.index = df[df[var].isnull()].index
    #df.loc[df[var].isnull(),var+'rand'] = randomsample
    

In [ ]:
whole(df,'onpromotion')
whole(df,'transactions')

In [ ]:
df.isnull().sum()

In [ ]:
desvar = [var for var in numer if len(df[var].unique())< 25 ]
desvar

In [ ]:
continous = [var for var in numer if var not in desvar and var not in   ['id']]
continous

In [ ]:
def fighist(df,var):
     df[var].hist(bins = 20)

In [ ]:
plt.figure(figsize = (10,10))
sns.set()
for i,var in enumerate (desvar):
    plt.subplot(4,3,i+1)
    fighist(df,var)

In [ ]:
plt.figure(figsize = (10,10))
sns.set()
for i,var in enumerate (continous):
    plt.subplot(4,4,i+1)
    fighist(df,var)

In [ ]:
import scipy.stats as stat
import pylab
def pltdata(df,var):
    plt.figure(figsize = (5,4))
    plt.subplot(1,2,1)
    df[var].hist()
    plt.subplot(1,2,2)
    stat.probplot(df[var],dist = 'norm',plot = pylab)
    

In [ ]:
for i,var in enumerate(desvar):
    
    pltdata(df,var)

In [ ]:
for i,var in enumerate(continous):
    
    pltdata(df,var)

In [ ]:
def logtran(df,var):
    df[var] = np.log(df[var])

In [ ]:
for var in continous:
    if 0 in df[var].unique():
        pass
    else:
        logtran(df,var)

In [ ]:
##again see their QQ plot

In [ ]:
for i,var in enumerate(continous):
    
    pltdata(df,var)

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
def fig(df,var):
   sns.boxplot(var,data = df)

In [ ]:
plt.figure(figsize = (10,5))
sns.set()
for i,var in enumerate (desvar):
    plt.subplot(8,2,i+1)
    fig(df,var)

In [ ]:
plt.figure(figsize = (10,5))
sns.set()
for i,var in enumerate (continous):
    plt.subplot(8,2,i+1)
    fig(df,var)

In [ ]:
### here outliears in only onpromotion and it is continous so here method additional of desvar

In [ ]:
def continu(df,var):
  upper=df[var].mean()+3*df[var].std()
  lower = df[var].mean()-3*df[var].std()
  iqr = df[var].quantile(.75) -df[var].quantile(.25)
  lbd = df[var].quantile(.25)-(iqr*3)
  ubd = df[var].quantile(.75)+(iqr*3)
  df.loc[df[var]>=ubd,var]=ubd
  print(upper)
  print(lower)
  print(iqr)
  print (lbd)
  print(ubd)

In [ ]:
for var in continous: 
  continu(df,var)

In [ ]:
plt.figure(figsize = (10,5))
sns.set()
for i,var in enumerate (continous):
    plt.subplot(8,2,i+1)
    fig(df,var)

In [ ]:
### same process do for train1sales. check graph then outliears and then remove. as it is descrete. check it is desvar or continous. 

In [ ]:
fighist(train1,'sales')

In [ ]:
fig(train1,'sales')

In [ ]:
 continu(train1,'sales')

In [ ]:
fig(train1,'sales')

In [ ]:
### above prpcess was to remove outliers in sales. but in original data. beacuse it is target.

In [ ]:
catvar = df.select_dtypes(include = 'object') 
catvar


In [ ]:
df.head()

In [ ]:
catvarfill = [var for var in catvar if catvar[var] .isnull().sum()>0]
catvarfill

In [ ]:
for var in catvarfill:
   catvar[var].fillna(0,inplace = True)
   


In [ ]:
df['type_x'].dtype

In [ ]:
for var in catvarfill:
 df[var] = df[var].astype(str)

In [ ]:
df['type_x'].dtype

In [ ]:
df.update(catvar)
df.isnull().sum()

In [ ]:
y = train1['sales']

In [ ]:
df['sales'] = y

In [ ]:
df.head(6)

In [ ]:
x = df.groupby('family')['sales'].mean().sort_values().index
x

In [ ]:
x = df.groupby('type_x')['sales'].mean().sort_values().index
x

In [ ]:
df['type_x'] = np.where(df['type_x']=='Holiday',1,0)

In [ ]:
x = df.groupby('locale')['sales'].mean().sort_values().index
x

In [ ]:
df['locale'] = np.where(df['locale']=='National',1,0)

In [ ]:
x = df.groupby('locale_name')['sales'].mean().sort_values().index
x

In [ ]:
df['locale_name'] = np.where(df['locale_name']=='Ecuador',1,0)

In [ ]:

x = df.groupby('description')['sales'].mean().sort_values().index
x

In [ ]:
top = ['Primer dia del ano','Carnaval']
for label in top:
 df[label] = np.where(df['description']==label,1,0)
### now drop description columns

In [ ]:
x = df.groupby('transferred')['sales'].mean().sort_values().index
x

In [ ]:
x = df.groupby('city')['sales'].mean().sort_values().index
x

In [ ]:
x = df.groupby('state')['sales'].mean().sort_values().index
x

In [ ]:
x = df.groupby('type_y')['sales'].mean().sort_values().index
x

In [ ]:
anotherone = ['family', 
        'city', 'state', 'type_y']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for var in anotherone :
 labels = le.fit_transform(df[var])
 df[var] = labels
df.head()

In [ ]:
### here days should be in the form of monday = 1 and tuesday = 2 and sunday = 7
eu = df.day.unique()
from sklearn.preprocessing import OrdinalEncoder
od = OrdinalEncoder(categories = [eu])
labels = od.fit_transform(df[['day']])
df['day'] = labels

In [ ]:
df.tail(60)

In [ ]:
transfer = pd.get_dummies(df['transferred'],drop_first = True)

In [ ]:
df1 = df.drop(columns =['transferred','date','description'],axis = 1)
df1 = pd.concat([df1,transfer],axis = 1)

In [ ]:
df1.isnull().sum()

In [ ]:
n = len(train1)
df2 = df1.drop(columns = 'sales',axis = 1)
Y = df1['sales']
xtrain = df2[:n]
xtest = df2[n:]
ytrain = Y[:n]
ytrain = ytrain.fillna(0)

In [ ]:
xtrain.isnull().sum()
ytrain.isnull().sum()

In [ ]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain,ytrain)
ypred = lr.predict(xtest)
ypredict = ypred.astype(int)
ypredict.shape

In [ ]:
from sklearn.linear_model import Ridge
rd = Ridge()
rd.fit(xtrain,ytrain)
ypred1 = rd.predict(xtest)
YP1 = ypred1.astype(int)


In [ ]:
from sklearn.linear_model import Lasso

ls = Lasso()
ls.fit(xtrain,ytrain)
ypred2 = ls.predict(xtest)


In [ ]:
from sklearn.ensemble import BaggingRegressor ,GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rancl = RandomForestRegressor(n_estimators= 100,criterion = 'mse',max_features = 'sqrt',min_samples_leaf = 10,random_state= 100)
rancl.fit(xtrain,ytrain)
ybest22 = rancl.predict(xtest)
ybest22 = ybest22.astype(int)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rancl = RandomForestRegressor(n_estimators= 100,criterion = 'mse',max_features = 'sqrt',min_samples_leaf = 10,random_state= 100)
rancl.fit(xtrain,ytrain)
ybest22 = rancl.predict(xtest)
ybest22 = ybest22.astype(int)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rancl = RandomForestRegressor(n_estimators= 100,criterion = 'mse',max_features = 'sqrt',min_samples_leaf = 10,random_state= 100)
rancl.fit(xtrain,ytrain)
ybest22 = rancl.predict(xtest)
ybest22 = ybest22.astype(int)

In [ ]:
import xgboost as xgb
xgbr=xgb.XGBRegressor()

In [ ]:
xgbtrain = xgb.DMatrix(xtrain)
xgbtest = xgb.DMatrix(xtest)
#yxgb =    xgb.DMatrix(ytrain)

In [ ]:
#xgbr.fit(xtrain,ytrain)
#ypredxgb = xgbr.predict(xgbtest)
#ypredxgb

In [ ]:
#### from here start hyperparametertununig


In [ ]:
## Hyper Parameter Optimization for randomforest regressor

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from datetime import datetime
d1 = datetime.now()

In [ ]:
def timer(start_time = None):
  if not start_time :
     start_time = datetime.now()
     return start_time
  elif start_time:
    thour,temp_sec= divmod((datetime.now()-start_time).total_seconds(),3600)
    tmin,tsec = divmod(temp_sec,60)
    print('\n'%(thour,tmin,round(tsec,2)))

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model


In [ ]:
#start_time = timer(None)
#rf_random.fit(xtrain, ytrain)
#timer(start_time)

In [ ]:
#### try to use lstm model

In [ ]:
### I should do work with hyperturning of randomforest beacuse in case of xgboost required to remove error or same features names 

In [ ]:
from sklearn.model_selection import KFold
kfold_validation = KFold(10)

In [ ]:
from sklearn. model_selection import cross_val_score
result = cross_val_score(rancl,xtrain,ytrain,cv = kfold_validation)
print(result)
print(np.mean(result))

In [ ]:
submittest = pd.concat([test['id'],pd.DataFrame(ybest22)],axis=1)
submittest.columns = ['id','sales']
submittest.to_csv('sample_submission.csv',index = False)
submittest